In [1]:
import os
import datetime
import glob
import csv
import math
import numpy as np
import pandas as pd
from google.colab import drive, files

# File locations
dir = '/content/drive/MyDrive/char_gender/'
data_dir = os.path.join(dir, 'data')

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
qualtrics_clean = pd.read_csv(os.path.join(data_dir, 'char_gender_final.csv'), header=1).iloc[1:].rename(columns={'Recorded Date': 'date',
                                            'If you agree to these conditions, please click "I consent to participate" below. If you do not agree, click the “I do not consent to participate” option.\n\nBy agreeing to participate, you confirm that you are over 18 years of age.\n\n \n\nI have read the above information.':'consent',
                                            'Your gender - Selected Choice':'gender',
                                            'Suppose you had both novels in your hand. Which one would you continue reading?': 'chosen_story'})
len(qualtrics_clean)

3508

In [4]:
prolific_w = pd.read_csv(os.path.join(data_dir, 'prolific_export_women.csv'))[['Participant id', 'Status', 'Started at']]
print(len(prolific_w))
prolific_m = pd.read_csv(os.path.join(data_dir, 'prolific_export_men.csv'))[['Participant id', 'Status', 'Started at']]
print(len(prolific_m))

1578
1605


In [5]:
proqual_w = prolific_w.merge(right=qualtrics_clean, left_on='Participant id', right_on='PROLIFIC_PID')
print(len(proqual_w))

1557


In [6]:
proqual_m = prolific_m.merge(right=qualtrics_clean, left_on='Participant id', right_on='PROLIFIC_PID')
len(proqual_m)

1589

In [7]:
def recent(_row):

  if '2024-05' in _row['date'] and int(_row['date'].split('-')[2][:2]) > 16:

    return True

In [8]:
def complete(_row):

  if not pd.isna(_row['chosen_story']) and not pd.isna(_row['gender']):

    return True

In [9]:
def correct(_row):

  cond_hike = (_row['In Novel A, how long is the trail that Sam takes?'] == '6 miles') and (_row['In Novel A, what plant does Sam rest against?'] == 'Juniper tree') and (_row['In Novel B, when does Alex go to the cafe?'] == 'Morning') and (_row['In Novel B, what is the material of the chair Alex sketches?'] == 'Wood')
  cond_coffee = (_row['In Novel B, how long is the trail that Sam takes?'] == '6 miles') and (_row['In Novel B, what plant does Sam rest against?'] == 'Juniper tree') and (_row['In Novel A, when does Alex go to the cafe?'] == 'Morning') and (_row['In Novel A, what is the material of the chair Alex sketches?'] == 'Wood')

  if cond_hike or cond_coffee:

    return True

In [10]:
# def check_incomplete_or_incorrect(row):

#   recent = '2024-05' in row['date'] and int(row['date'].split('-')[2][:2]) > 16

#   cond_hike2 = (row['In Novel A, how long is the trail that Sam takes?'] != '6 miles') or (row['In Novel A, what plant does Sam rest against?'] != 'Juniper tree') or (row['In Novel B, when does Alex go to the cafe?'] != 'Morning') or (row['In Novel B, what is the material of the chair Alex sketches?'] != 'Wood')
#   cond_coffee2 = (row['In Novel B, how long is the trail that Sam takes?'] != '6 miles') or (row['In Novel B, what plant does Sam rest against?'] != 'Juniper tree') or (row['In Novel A, when does Alex go to the cafe?'] != 'Morning') or (row['In Novel A, what is the material of the chair Alex sketches?'] != 'Wood')
#   incorrect = cond_hike2 or cond_coffee2

#   incomplete = pd.isna(row['chosen_story']) or pd.isna(row['gender'])

#   if recent and incomplete and incorrect:

#     return True

# Prolific check

#### Check proportion of treatments

In [11]:
def check_treatments(df, extra_cond):

  H = 0
  C = 0
  W = 0
  M = 0
  H_W = 0
  H_M = 0
  C_W = 0
  C_M = 0

  for index, row in qualtrics_clean.iterrows():

      if row['First Story'] == 'Hike':
        H += 1

      if row['First Story'] == 'Coffee':
        C += 1

      if row['First Gender'] == 'W':
        W += 1

      if row['First Gender'] == 'M':
        M += 1

      if row['First Story'] == 'Hike' and row['First Gender'] == 'W':
        H_W += 1

      elif row['First Story'] == 'Hike' and row['First Gender'] == 'M':
        H_M += 1

      elif row['First Story'] == 'Coffee' and row['First Gender'] == 'W':
        C_W += 1

      elif row['First Story'] == 'Coffee' and row['First Gender'] == 'M':
        C_M += 1

  print(f'First story Hike: {H}\n First story Coffee: {C}\n First Character Gender W: {W}\n First Character Gender M: {M}\n\n',
        f'First Hike and Woman: {H_W}\n First Hike and Man: {H_M}\n First Coffee and Woman: {C_W}\n First Coffee and Man: {C_M}\n')

In [12]:
awesome_PIDs = check_treatments(qualtrics_clean, True)

First story Hike: 1744
 First story Coffee: 1745
 First Character Gender W: 1746
 First Character Gender M: 1743

 First Hike and Woman: 868
 First Hike and Man: 876
 First Coffee and Woman: 878
 First Coffee and Man: 867



#### Approved are incomplete and incorrect?

In [13]:
def check_incomplete_incorrect(df, status):

  for index, row in df.iterrows():

    if row['Status'] == status:

      if status == 'APPROVED' or status == 'AWAITING REVIEW':
        if not complete(row):
          print(status, 'but incomplete', row['PROLIFIC_PID'])

        if not correct(row):
          print(status, 'but incorrect', row['PROLIFIC_PID'])

      elif status == 'RETURNED':
        if complete(row) and correct(row):
          print(status, 'but complete and correct', row['PROLIFIC_PID'])

In [14]:
check_incomplete_incorrect(proqual_w, 'APPROVED')

APPROVED but incomplete 5c9623dd35599200175642e4
APPROVED but incorrect 5c9623dd35599200175642e4
APPROVED but incomplete 65e8cf8f4c7424fa062e54a3
APPROVED but incorrect 65e8cf8f4c7424fa062e54a3


In [15]:
check_incomplete_incorrect(proqual_m, 'APPROVED')

APPROVED but incomplete 5fd0ef1dbfdf0801f669f642
APPROVED but incorrect 5fd0ef1dbfdf0801f669f642
APPROVED but incomplete 643883e97bc73c8245bc31d3
APPROVED but incorrect 643883e97bc73c8245bc31d3
APPROVED but incomplete 663a8ea83cc4d5845ef3a672
APPROVED but incorrect 663a8ea83cc4d5845ef3a672
APPROVED but incomplete 572d05c688902e00091e4a2c
APPROVED but incorrect 572d05c688902e00091e4a2c
APPROVED but incomplete 59296fb2e8a7ff000163f033
APPROVED but incorrect 59296fb2e8a7ff000163f033
APPROVED but incomplete 63ecfea248ea89e9bb6b2a9a
APPROVED but incorrect 63ecfea248ea89e9bb6b2a9a
APPROVED but incomplete 6361b032a20001fc86b0ff1b
APPROVED but incorrect 6361b032a20001fc86b0ff1b
APPROVED but incomplete 616f727b03e888e0f8213eec
APPROVED but incorrect 616f727b03e888e0f8213eec
APPROVED but incomplete 591d369e88bbb500013a8517
APPROVED but incorrect 591d369e88bbb500013a8517
APPROVED but incomplete 5ca8ffbec942ab0016cfe9d2
APPROVED but incorrect 5ca8ffbec942ab0016cfe9d2
APPROVED but incomplete 662caf

#### Awaiting review is incomplete or incorrect?

In [16]:
check_incomplete_incorrect(proqual_w, 'AWAITING REVIEW')

In [17]:
check_incomplete_incorrect(proqual_m, 'AWAITING REVIEW')

#### Returned responses but complete and correct?

In [18]:
def check_complete_correct(df):

  for index, row in df.iterrows():

    if row['Status'] == 'RETURNED' and complete(row) and correct(row):

      print('Returned but complete and correct', row['PROLIFIC_PID'])

In [19]:
check_complete_correct(proqual_w)

Returned but complete and correct 663287eff8f3dda45d8a2125


In [20]:
check_complete_correct(proqual_m)

#### Approved but missing

In [21]:
approved_pids_w = prolific_w[prolific_w['Status'] == 'APPROVED'][['Participant id']].values
len(approved_pids_w)

1497

In [22]:
approved_pids_m = prolific_m[prolific_m['Status'] == 'APPROVED'][['Participant id']].values
len(approved_pids_m)

1495

In [23]:
awesome_pids = []
awesome_valid_pids = []
awesome_pids_W = []
awesome_pids_M = []
awesome_pids_O = []

for index, row in qualtrics_clean.iterrows():

  if recent(row) and complete(row) and correct(row):
    awesome_pids.append(row['PROLIFIC_PID'])

    if row['gender'] == 'Woman':
      awesome_pids_W.append(row['PROLIFIC_PID'])
      awesome_valid_pids.append(row['PROLIFIC_PID'])
    elif row['gender'] == 'Man':
      awesome_pids_M.append(row['PROLIFIC_PID'])
      awesome_valid_pids.append(row['PROLIFIC_PID'])
    else:
      awesome_pids_O.append(row['PROLIFIC_PID'])

print(len(awesome_pids), len(awesome_valid_pids), len(awesome_pids_W), len(awesome_pids_M), len(awesome_pids_O))
len(set(awesome_pids))

3002 2985 1493 1492 17


3002

In [24]:
# How many approved are missing W
missing_w = np.setdiff1d(np.array(approved_pids_w), np.array(awesome_pids))
len(missing_w), missing_w

(1, array(['60e2577f8c52db9d1fb5ffac'], dtype=object))

In [25]:
# How many approved are missing M
missing_m = np.setdiff1d(np.array(approved_pids_m), np.array(awesome_pids))
len(missing_m), missing_m

(11,
 array(['6135752aa4ee5158b0708670', '616f727b03e888e0f8213eec',
        '64050cfd151bc9945fd3eac9', '6406d015ad2a96ceef1f86cc',
        '643883e97bc73c8245bc31d3', '6464b45cf8e8a0f06fe011b9',
        '657d9d56eb56655941d8483f', '6603d47c436bc4cbabc108bc',
        '661017f723983f48d9f730e1', '661565a9e2816f002d25ea56',
        '662caff5542404739fbbb92a'], dtype=object))

#### Awesome but not approved

In [26]:
prolific_w2 = pd.read_csv(os.path.join(data_dir, 'prolific_export_women2.csv'))[['Participant id', 'Status', 'Started at']]
print(len(prolific_w2))
approved_pids_w2 = prolific_w2[prolific_w2['Status'] == 'APPROVED'][['Participant id']].values
len(approved_pids_w2)

5


5

In [27]:
prolific_m2 = pd.read_csv(os.path.join(data_dir, 'prolific_export_men2.csv'))[['Participant id', 'Status', 'Started at']]
print(len(prolific_m2))
approved_pids_m2 = prolific_m2[prolific_m2['Status'] == 'APPROVED'][['Participant id']].values
len(approved_pids_m2)

18


16

In [28]:
approved_all_w = np.append(approved_pids_w, approved_pids_w2)
approved_all_m = np.append(approved_pids_m, approved_pids_m2)
approved_all = np.append(approved_all_w, approved_all_m)

In [29]:
np.setdiff1d(np.array(awesome_pids), approved_all)

array(['663287eff8f3dda45d8a2125'], dtype='<U24')

## Check unfinished submissions

In [33]:
unfinished_clean = pd.read_csv(os.path.join(data_dir, 'char_gender_unfinished.csv'), header=1).iloc[1:].rename(columns={'Recorded Date': 'date',
                                            'If you agree to these conditions, please click "I consent to participate" below. If you do not agree, click the “I do not consent to participate” option.\n\nBy agreeing to participate, you confirm that you are over 18 years of age.\n\n \n\nI have read the above information.':'consent',
                                            'Your gender - Selected Choice':'gender',
                                            'Suppose you had both novels in your hand. Which one would you continue reading?': 'chosen_story'})
len(unfinished_clean)

156

In [34]:
unfinished_pids = unfinished_clean.PROLIFIC_PID.unique()

In [46]:
len(np.intersect1d(np.array(proqual_w['PROLIFIC_PID']), np.array(unfinished_pids)))

59

In [47]:
len(np.intersect1d(np.array(proqual_m['PROLIFIC_PID']), np.array(unfinished_pids)))

97

In [49]:
len(np.intersect1d(np.array(proqual_w[proqual_w['Status'] == 'APPROVED']['PROLIFIC_PID']), np.array(unfinished_pids)))

2

In [52]:
len(np.intersect1d(np.array(proqual_m[proqual_m['Status'] == 'APPROVED']['PROLIFIC_PID']), np.array(unfinished_pids)))

10